In [2]:
import svv_connector as svv   # SEE BELOW FOR ITS DEFINITION

Extend SVVConnection class to be able to execute a query ad get results as following:

(1) Add a method called executeQuery that accepts a cursor and a query (as str). It checks if cursor is None, and if it is None, it prints an error message and returns. Otherwise it executes the query that is passed to the method on the passed cursor.

(2) Add another method called getBatchQueryResults that accepts batch_size (which defaults to 1), and using try-except block and a generator returns a batch_size of the query results. Have a way to stop the generator when all results are returned.

(3) Test your work with a simple query!

In [13]:
class SVVConnection(object):
    
    def __init__(self):
        ''' Opens connection on Initialisation
        '''
        self.Connection = svv.get_svv_connection()
    
    def close(self):
        ''' Closes connection to svv
        '''
        try:
            self.my_cursor
        except:    
            self.my_cursor = None
            
        if self.my_cursor is not None:    
            self.my_cursor.close()
            
        self.Connection.close()
    
    def reOpen(self):
        ''' Re-opens connection to svv
        '''
        if self.Connection.closed:
            self.Connection = svv.get_svv_connection()
    
    def getCursor(self):
        ''' obtains a new cursor
        '''
        self.my_cursor = self.Connection.cursor()
        return self.my_cursor
    
    def closeCursor(self):
        ''' Closes a cursor if one is open
        '''
        try:
            self.my_cursor
        except:
            self.my_cursor = None
        
        if self.my_cursor is not None:
            self.my_cursor.close()
            
    def executeQuery(self, my_cursor, query):
        ''' Executes a query on SVV using an opened cursor my_cursor
        '''
        
        if my_cursor is None:
            print ("Cursor not open. Call getCursor first.")
            return
        
        try:
            my_cursor.execute(query)
        except:
            print ("Error executing query. Is the cursor opened?")
            
    def getBatchQueryResults(self, batch_size=1):
        ''' Returns a generator from which query results can be sources in batch_size
        '''
        res = 0
        
        try:
            while res != []:
                res = cur.fetchmany(batch_size)
                yield res
                
        except:
            print ("Error fetching query results.")

In [16]:
my_connection = SVVConnection()

In [17]:
cur = my_connection.getCursor()

In [18]:
query = "Select Top 10 * from sandbox.viewers_summary_hc_2018c_es;"

In [19]:
my_connection.executeQuery(cur, query)

In [21]:
my_results = my_connection.getBatchQueryResults(10)

In [23]:
print (next(my_results))

[('77ff5199-834a-4d61-ab95-0a0bf9e23448', 'M', 65, 'ad.axa.hc.me', 'TN38 9', 'ST. LEONARDS-ON-SEA', 174, 'Inactive', '2', '07', '00', '1', '1', '11', '3', '06', '2', '0', '1', '3', 38400, 'D', '3', 0, 0, 0, 0, '[{"id":"DOCUMENTARY","ranking":"1"}]', Decimal('617.9450000000'), None, Decimal('1.0000'), 'TN38_9', 'TN38_9', 'Hard-pressed living', 'Migration and churn', 'Young hard-pressed families', 1, None), ('c5477e47-985a-4da0-8bc9-e6588c889ece', 'M', 39, 'ad.me', 'NG6 9', 'NOTTINGHAM', 40, 'Active', '2', '07', '03', '1', '0', '06', '3', '06', '2', '0', '0', '4', 45300, 'D', '3', 0, 0, 0, 0, '[{"id":"CRIME DOCUMENTARY"}]', None, Decimal('2796.3560000000'), Decimal('0.0000'), 'NG6_9', 'NG6_9', 'Multicultural metropolitans', 'Rented family living', 'Social renting young families', 0, None), ('c7ede143-4d08-4637-9da6-e50d88767e9c', 'M', 53, 'ad.axa.me', 'BA4 6', 'SHEPTON MALLET', 50, 'Active', '3', '07', '00', '1', '0', '04', '2', '09', '3', '0', '1', '4', 45000, 'I', '4', 0, 0, 0, 0, '[{"

In [26]:
print (next(my_results))

Error fetching query results.


StopIteration: 

In [27]:
my_connection.closeCursor()

In [28]:
my_connection.close()

### _/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/_/

import psycopg2


def get_svv_connection():

    con=psycopg2.connect(dbname= 'svv', host='svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                         port= 5439, user= 'my_user_name', password= 'my_password')
    
    return con